<a href="https://colab.research.google.com/github/marchelo2212/curso_llm/blob/main/notebooks/est_notebook_0_introduccion_langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Escuela Colombiana de Ingeniería Julio Garavito

## Taller de Inteligencia Artificial Aplicada a la Educaicón - Avanzado

**Facilitador:** Marcelo Sotaminga

**OCT-2025**


Estimado participante, dentro de este cuaderno se presnetarán una serie de ejercicios articulas a que usted conozca el framework LangChain, su potencialidad y manera de uso desde un enfoque técnico.

El Objetivo que se persigue con este taller es:

Lograr conocimientos sólidos sobre langchain para la elaboración de un proyecto/prototipo de artefacto tecnológico que emplee un LLM y que atienda una necesidad particulr del participante.

Para alcanzar este objetivo se desarrollarán 4 cuadernos de google colab con esstos detalles:

1. Cuaderno 0 — Introducción a LangChain y fundamentos prácticos de RAG
2. Notebook 1: Chatbot con FAQ /QA
3. Notebook 2: Chatbot RAG
4. Notebook 3: Chatbot RAG con memoria




# Cuaderno 0 — Introducción a LangChain y fundamentos prácticos de RAG

**Duración sugerida:** 45–60 min  
**Objetivo:** activar conocimientos previos sobre LLMs y conectar los componentes básicos de LangChain que luego se usarán en las prácticas de **FAQ Bot** y **Chatbot RAG**.

> **Requisitos:** Cuenta en Google Colab con acceso a Internet y una clave de API de OpenAI (o usa un modelo local/alternativo si lo prefieres).

## Agenda del cuaderno
1. Preparación del entorno (instalación y claves)
2. Primer ejemplo: llamada simple a un modelo vía LangChain
3. PromptTemplate + LLMChain (encadenando un prompt)
4. Carga de documentos (PDF) con `DocumentLoader`
5. División del texto (chunking) con `TextSplitter`
6. Introducción rápida a embeddings
7. Mini‑ejercicio de comprensión
8. Conclusión y siguiente paso

## 1) Preparación del entorno
Ejecuta la celda siguiente para instalar dependencias mínimas. En Colab tardará ~1–3 minutos.

**Paquetes a instalar**
- `langchain` y `langchain-openai`: orquestación y wrapper de OpenAI para LangChain.
- `chromadb`: vector store local liviano (se usará en el siguiente cuaderno de RAG).
- `tiktoken`: tokenizador útil para contar tokens.
- `pypdf`: extracción de texto desde PDF.


In [ ]:
# ⬇️ Instalar dependencias mínimas
!pip install -qU \
    langchain \
    langchain-openai \
    chromadb \
    tiktoken \
    pypdf \
    scikit-learn \
    matplotlib \
    numpy

### Clave de API (OpenAI)
En Colab puedes guardar tu clave de forma temporal:
- Ve a `Entorno de ejecución` → `Restablecer estado de fábrica` si es necesario.
- Usa la celda siguiente y **sustituye** `TU_API_KEY` por tu clave real.

> ⚠️ No compartas tu API key.

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'APIKEY'  # ← Reemplaza por tu clave (o usa otro proveedor en los cuadernos siguientes)
print('API key configurada (longitud):', len(os.environ.get('OPENAI_API_KEY','')))

## 2) Primer ejemplo: pregunta directa al modelo
Mostramos cómo LangChain abstrae el uso de un LLM.

> **Reflexión:** Este enfoque **no** tiene memoria, ni grounding, ni control fino del contexto.

In [ ]:
from langchain_openai import ChatOpenAI

# Temperatura baja para mayor determinismo en ejemplos docentes
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
resp = llm.invoke("Explica en 3 líneas qué es la inteligencia artificial generativa.")
print(resp.content)


### 2.1) Ejercicio práctico
Agregue aquí el código para ejecutar una pregunta diferente a la ubicada en el cuaderno.
Aquí `modifica la pregunta`, intenta lograr que el `modelo falle o responda equivocadamente`.


In [ ]:
#Aquí tu código





### 2.2) Ejercicio práctico
Agregue aquí el código para ejecutar una pregunta diferente a la ubicada en el cuaderno.
Aquí modifica el `modelo de OpenAI`, investiga cuáles son los modelos que existen. Cambia por alguno que halles, responde ¿Ha funcionado la consulta? ¿Su respuesta ha cambiado? ¿En caso que no haya funcionado, por qué pasa esto?


In [ ]:
#Aquí tu código

Agrega aquí tus respuestas a la pregunta.


> Respuestas...



## 3) PromptTemplate + LLMChain
Un *PromptTemplate* permite definir una plantilla con **variables**. Con `LLMChain` unimos el modelo y el prompt, creando un bloque reutilizable.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
template = (
    """
Eres un profesor de ingeniería. Resume el siguiente concepto en lenguaje técnico y claro.
Incluye un ejemplo breve. Limita la respuesta a 4 líneas.
Concepto: {concepto}
"""
)
prompt = PromptTemplate(template=template, input_variables=["concepto"])
chain = prompt | llm

print(chain.invoke({"concepto": "aprendizaje supervisado"}))


## 3.1) Ejercicio práctico
Modifica aquí el código y adecua a tu área de conocimiento.


In [ ]:
#Aquí tu código



## 3.2) Ejercicio práctico
Agrega la posibilidad que se modifique el modelo de IA usando en la consulta.


In [ ]:
#Aquí tu código


## 3.3) Ejercicio práctico
Modifica el template del prompt de tal manera que el usuario ingrese:
1. Área del conocimiento.
2. Concepto

In [ ]:
#Aquí tu código

## 4) Carga de documentos (PDF) con `DocumentLoader`
Para esta demostración subiremos un **PDF corto**. En Colab:
1) Ejecuta la celda, 2) Elige un PDF desde tu PC, 3) Confirma el nombre del archivo cargado.

In [ ]:
from google.colab import files
uploaded = files.upload()  # ← Selecciona un PDF corto (1–3 páginas)
uploaded_files = list(uploaded.keys())
uploaded_files[:3]

In [ ]:
#!pip install -U langchain_community
from langchain_community.document_loaders import PyPDFLoader

pdf_path = uploaded_files[0]
loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f"Páginas extraídas: {len(docs)}")
print("\nVista previa de la primera página (primeros 500 chars):\n")
print(docs[0].page_content[:500])

## 5) División del texto (chunking)
Dividimos el documento en fragmentos manejables. Esto prepara el material para **retrieval** y evita desbordar la ventana de contexto del LLM.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # tamaño del fragmento (ajustable)
    chunk_overlap=100,   # solape entre fragmentos para mantener continuidad
)
chunks = splitter.split_documents(docs)
print(f"Total de fragmentos: {len(chunks)}")
print("\nPrimer fragmento:\n", chunks[0].page_content[:400])

### 5.1 Investiga y responde
**¿Por qué es importante hacer `chunl_overlap`?**


> Respuesta

**¿Cuál es el tamaño ideal del chunk_size? ¿Por qué?**


> Respuesta


## 6) Introducción rápida a embeddings
Un *embedding* es una representación numérica (vector) del significado de un texto. Nos permite medir **similitud semántica** entre fragmentos y consultas.

Ejecuta la siguiente celda para generar un embedding de ejemplo y observar su dimensión.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Try a different model that might be available
embedder = OpenAIEmbeddings(model="text-embedding-3-small")
vec = embedder.embed_query("mecánica de fluidos")
print("Dimensión del embedding:", len(vec))
print("Primeros 8 valores:", [round(v, 4) for v in vec[:8]])

Aquí compararemos la similitud semántica del embedding "mecánica de fluidos" con otros dos "Ingeniería hidráulica" y "Receta de pasteles"

In [ ]:
# 1. Instalación
!pip install -qU scikit-learn

# 2. Importaciones
from langchain_openai import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

# 3. Configurar el embedder
embedder = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. Crear los vectores
print("Generando embeddings...")
vec_1 = embedder.embed_query("mecánica de fluidos")
vec_similar = embedder.embed_query("ingeniería hidráulica")
vec_diferente = embedder.embed_query("receta de pastel")
print("Dimensión (vec_1):", len(vec_1))


# 5. Calcular la similitud
# Nota: cosine_similarity espera listas de vectores (2D), por eso usamos [vec_1]
sim_1_vs_similar = cosine_similarity([vec_1], [vec_similar])[0][0]
sim_1_vs_diferente = cosine_similarity([vec_1], [vec_diferente])[0][0]

# 6. Mostrar resultados
print("\n--- Resultados de Similitud (1.0 es idéntico) ---")
print(f"Fluidos vs. Hidráulica: {sim_1_vs_similar:.4f}")
print(f"Fluidos vs. Pastel:     {sim_1_vs_diferente:.4f}")

In [ ]:
# 1. Explicitly install a compatible version of scikit-learn and dependencies


# 2. Importaciones
from langchain_openai import OpenAIEmbeddings
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# 3. Lista de textos a comparar
textos = [
    "mecánica de fluidos",   # Ingeniería
    "ingeniería hidráulica", # Ingeniería (similar)
    "aerodinámica",          # Ingeniería (relacionado)
    "flujo laminar",         # Ingeniería (concepto específico)
    "receta de pastel",      # Cocina
    "literatura clásica",    # Humanidades
    "filosofía"              # Humanidades (similar)
]

# 4. Configurar y generar embeddings (de las palabras)
print("Generando embeddings (1536-D)...")
# Suponiendo que el embedder esté definido en otro lugar.
embedder = OpenAIEmbeddings(model="text-embedding-3-small") # Assuming you have access to this model
vectores = embedder.embed_documents(textos)

# 5. Reducción de Dimensionalidad (Necesaria para poder graficar)
print("Reduciendo dimensiones de 1536-D a 2-D con PCA...")

# Convertir la lista de vectores a un formato que PCA entienda (NumPy array)
X = np.array(vectores)

# Configurar PCA para reducir a 2 componentes (dimensiones)
pca = PCA(n_components=2)

# Ejecutar la reducción
vectores_2d = pca.fit_transform(X)

# 6. Graficar los resultados
print("Graficando los vectores en 2D...")
plt.figure(figsize=(10, 7))
x = vectores_2d[:, 0]  # Coordenada X
y = vectores_2d[:, 1]  # Coordenada Y

# Dibuja los puntos
plt.scatter(x, y, alpha=0.7)

# Añade etiquetas a cada punto
for i, txt in enumerate(textos):
    plt.annotate(txt, (x[i], y[i]), xytext=(5, 2), textcoords='offset points')

plt.title("Visualización de Embeddings (Reducidos a 2D con PCA)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.grid(True)
plt.show()

## 7) Mini‑ejercicio de comprensión
Responde en una celda Markdown o en comentarios de código:
1. ¿Qué pasos del pipeline RAG ya realizamos aquí?  
2. ¿Por qué el *chunking* ayuda a mejorar la recuperación y la calidad de respuesta?  
3. ¿Qué limitaciones tiene usar solo el flujo directo `Prompt → LLM` sin retrieval?


Aquí tus respuestas


> Añadir blockquote



## 8) Conclusión y siguiente paso
- Ya conoces los *building blocks* esenciales de LangChain: `DocumentLoader`, `TextSplitter`, `Embeddings` y un primer `LLMChain`.
- En el próximo cuaderno implementaremos un **FAQ Bot** (sin retrieval) y luego un **Chatbot RAG** con recuperación semántica y citación de fuentes.

### Referencias
- P. Lewis *et al.*, “Retrieval‑Augmented Generation for Knowledge‑Intensive NLP Tasks,” NeurIPS 2020. https://arxiv.org/abs/2005.11401
- LangChain Docs — Tutorial RAG: https://python.langchain.com/docs/tutorials/rag/
- LangChain Docs — Knowledge base: https://docs.langchain.com/oss/python/langchain/knowledge-base
